In [ ]:
from d123.dataset.scene.scene_builder import ArrowSceneBuilder
from d123.dataset.scene.scene_filter import SceneFilter

from nuplan.planning.utils.multithreading.worker_sequential import Sequential
from d123.common.datatypes.sensor.camera import CameraType

In [ ]:
# split = "nuplan_private_test"
# log_names = ["2021.09.29.17.35.58_veh-44_00066_00432"]


splits = ["wopd_train"]
# log_names = None



# splits = ["nuplan_private_test"]
log_names = None

scene_tokens = None

scene_filter = SceneFilter(
    split_names=splits,
    log_names=log_names,
    scene_tokens=scene_tokens,
    duration_s=19,
    history_s=0.0,
    timestamp_threshold_s=20,
    shuffle=False,
    camera_types=[CameraType.CAM_F0],
)
scene_builder = ArrowSceneBuilder("/home/daniel/d123_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print(f"Found {len(scenes)} scenes")

In [ ]:
from typing import Tuple
import matplotlib.pyplot as plt
from d123.common.visualization.matplotlib.observation import add_box_detections_to_ax, add_default_map_on_ax, add_ego_vehicle_to_ax, add_traffic_lights_to_ax
from d123.common.visualization.matplotlib.plots import plot_scene_at_iteration
from d123.dataset.scene.abstract_scene import AbstractScene

def _plot_scene_on_ax(ax: plt.Axes, scene: AbstractScene, iteration: int = 0, radius: float = 80) -> plt.Axes:

    ego_vehicle_state = scene.get_ego_state_at_iteration(iteration)
    box_detections = scene.get_box_detections_at_iteration(iteration)

    point_2d = ego_vehicle_state.bounding_box.center.state_se2.point_2d
    # add_default_map_on_ax(ax, scene.map_api, point_2d, radius=radius, route_lane_group_ids=route_lane_group_ids)
    # add_traffic_lights_to_ax(ax, traffic_light_detections, scene.map_api)

    add_box_detections_to_ax(ax, box_detections)
    add_ego_vehicle_to_ax(ax, ego_vehicle_state)

    ax.set_xlim(point_2d.x - radius, point_2d.x + radius)
    ax.set_ylim(point_2d.y - radius, point_2d.y + radius)

    ax.set_aspect("equal", adjustable="box")
    return ax


def plot_scene_at_iteration(
    scene: AbstractScene, iteration: int = 0, radius: float = 80
) -> Tuple[plt.Figure, plt.Axes]:

    fig, ax = plt.subplots(figsize=(10, 10))
    _plot_scene_on_ax(ax, scene, iteration, radius)
    return fig, ax


scene_index = 0
plot_scene_at_iteration(scenes[scene_index], iteration=0,radius=80)